In [ ]:
import numpy as np
import xarray as xr

from src.tools import RiversHandler, GridHandler, OutputHandler

In [ ]:
river_ds = xr.open_dataset('fram_data/norfjords_160m_river_v3+.nc')
grid_ds = xr.open_dataset('fram_data/norfjords_160m_grid_v3+.nc')
rst_ds = xr.open_dataset('fram_data/roho160_rst.nc')

Check rivers

In [ ]:
RiversHandler.check_coords(river_ds, 562, 110)

In [ ]:
river_xi_eta_dir = RiversHandler.check_rivers_runoff(river_ds, verbose=False)
bad_rivers_numbers = [18]
bad_rivers = [river_xi_eta_dir[coord] for coord in bad_rivers_numbers]
print(bad_rivers)

In [ ]:
riv_plotter = RiversHandler(grid_ds)
riv_plotter.plot_rivers_luv(bad_rivers)

In [ ]:
grid_plotter = GridHandler(grid_ds)
grid_plotter.plot_h(562, 110, 10)
grid_plotter.plot_masks(562, 110, 5)

In [ ]:
output_plotter = OutputHandler(grid_ds)
u, v = output_plotter.plot_u_v(rst_ds, 562, 110, -1, 5)